<a href="https://colab.research.google.com/github/bhargav1237874/grandhe_INFO5731_spring2021/blob/main/Citation_context_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
from selenium import webdriver
import time
import requests
import os

# read the dataset

data=pd.read_excel('NLP_dataset_citation_context_edited.xlsx',sheet_name='Sheet2')

# read all the links

driver=webdriver.Chrome('chromedriver.exe')
all_items={}
for link,short_name in zip(data['links'],data['Shortname']):
    items={}
    driver.get(link)
    time.sleep(2)
    elem = driver.find_elements_by_css_selector('li.txt-md-sm > a')
    for element in elem:
        items[element.text]=element.get_attribute('href')    
    all_items[short_name]=items

# read link content

driver=webdriver.Chrome('chromedriver.exe')

for item,value in all_items.items():
    for item_child,value_child in value.items():
        driver.get(value_child)
        time.sleep(2)
        info_button=driver.find_element_by_xpath('/html/body/container/div/button')
        info_button.click()
        try:
            element=driver.find_element_by_partial_link_text('https://cite.case.law')
            value[item_child]=element.text
        except:
            value[item_child]=''

# create chunk of files to read

file = open("links.pickle",'rb')
object_file = pickle.load(file)

count=0
for key,value in object_file.items():
    count=count+1
    with open(str(count)+'links.pickle', 'wb') as handle:
        pickle.dump(value, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
file = open("links.pickle",'rb')
object_file = pickle.load(file)

# create start of file creation


In [ ]:
start=0
end=12

In [ ]:
for i in range(start,end+1):
    count=0
    print('started: '+str(i))
    file = open(str(i)+"links.pickle",'rb')
    object_file = pickle.load(file)
    object_file_copy=object_file.copy()
    print(object_file)
    driver=webdriver.Chrome('chromedriver.exe')
    for item_child,value_child in object_file.items():
        count=count+1
        if count<10:
            try:
                driver.get(value_child)
                time.sleep(5)
                object_file_copy[item_child]=driver.find_element_by_class_name('main-content').text
            except:
                object_file_copy[item_child]=''
        else:
            object_file_copy[item_child]=''
    if len(set(list(object_file_copy.values())))>1:
        with open(str(i)+'scapped_data.pickle', 'wb') as handle:
            pickle.dump(object_file_copy, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        print('stopped at: '+str(i))
        time.sleep(600)
    

started: 8
{'Naylor’s Administrator v. Moffatt': 'https://cite.case.law/mo/29/126/', 'Arambula v. Atwell': 'https://cite.case.law/sw2d/948/173/', 'Graham v. Ozark Mountain Sightseeing, Inc.': 'https://cite.case.law/f3d/181/924/', 'Singer v. Siedband': 'https://cite.case.law/sw3d/138/750/9224593/', 'Mikesic v. Trinity Lutheran Hosp.': 'https://cite.case.law/sw2d/980/68/'}
stopped at: 8
started: 9
{'Posey v. Com.': 'https://cite.case.law/sw3d/185/170/', 'Roberts ex rel. Inmates of California State Prison for Men v. State of California Department of Corrections': 'https://cite.case.law/f2d/183/580/', 'Austin v. United States': 'https://cite.case.law/f-supp/40/777/', 'State ex rel. Folsom Bros. v. Mayor of New Orleans': 'https://cite.case.law/la-ann/32/709/', 'Barringtons v. Logan’s Administrators': 'https://cite.case.law/ky/32/432/', 'State v. Claiborne': 'https://cite.case.law/tenn/19/331/', 'Marshall v. Donovan': 'https://cite.case.law/ky/73/681/', 'Hall v. United States': 'https://cite

AttributeError: 'str' object has no attribute 'copy'

# collating all the files

In [ ]:
all_data=[]
for i in range(start,end+1):
    try:
        file = open(str(i)+"scapped_data.pickle",'rb')
        object_file = pickle.load(file)
    except:
        pass
    data=(pd.DataFrame.from_dict(object_file,orient='index'))
    all_data.append(data)
pd.concat(all_data).to_excel('scrapped_data.xlsx')

In [ ]:
all_data=pd.read_excel('scrapped_data.xlsx')

In [ ]:
all_data.rename(columns={'Unnamed: 0':'col_1',0:'col_2'},inplace=True)

In [ ]:
all_data=all_data.dropna()

In [ ]:
all_data

,col_1,col_2
1,Bryant v. Whitcher,"Bryant v. Whitcher, 52 N.H. 158 (1872)\nJuly 1..."
2,Pool v. George,"Pool v. George, 30 Tenn. App. 608, 209 S.W.2d ..."
3,Gage v. Epperson,"Gage v. Epperson, 39 Tenn. 669, 2 Head 669 (18..."
4,Lazell v. Powell,"Lazell v. Powell, 1 Unrep. Tenn. Cas. 195 (185..."
5,Taylor v. Pope,"Taylor v. Pope, 45 Tenn. 413, 5 Cold. 413 (186..."
...,...,...
291,Lazell v. Powell,"Lazell v. Powell, 1 Unrep. Tenn. Cas. 195 (185..."
292,Taylor v. Pope,"Taylor v. Pope, 45 Tenn. 413, 5 Cold. 413 (186..."
293,Meador v. Meador,"Meador v. Meador, 50 Tenn. 562, 3 Heisk. 562 (..."
294,Hawkins v. Davis,"Hawkins v. Davis, 67 Tenn. 506 (1875)\nDec. 18..."


In [ ]:
file = open("links.pickle",'rb')
object_file = pickle.load(file)

In [ ]:
all_dataframe=[]
for key,value in object_file.items():
    all_items=[]
    for item_key,item_value in value.items():
        all_items.append(item_key)
    data=pd.DataFrame(all_items)
    data['col_3']=key
    all_dataframe.append(data)
    data.rename(columns={0:'col_1'},inplace=True)
final_data=pd.concat(all_dataframe)

In [ ]:
final_data

,col_1,col_3
0,Davis v. State,Ables v. Donley
1,"Ray v. Anesthesia Assocs. of Mobile, P.C.",Ables v. Donley
2,Lampsen v. Brander,Ables v. Donley
3,G. H. & S. A. R’y Co. v. Marsden,Ables v. Donley
4,Faulkner v. Territory,Ables v. Donley
...,...,...
18,Nance v. Piano Co.,Arendale v. Samuel D. Morgan & Co.
19,Ironton Garage Co. v. McKee,Arendale v. Samuel D. Morgan & Co.
20,Jones v. Ironton Garage Co.,Arendale v. Samuel D. Morgan & Co.
21,Robertson v. Wade,Arendale v. Samuel D. Morgan & Co.


In [ ]:
merged_data=pd.merge(all_data,final_data,how='right')

In [ ]:
merged_data['col_5']=merged_data['col_2'].apply(lambda x:str(x).split('\n'))

In [ ]:
def string_bool(list_all,item):
    for each_item in list_all:
        if item in each_item:
            return (True,each_item)
    return (False,'')

In [ ]:
merged_data['col_4']=merged_data.apply(lambda x:string_bool(x['col_5'],x['col_3']),axis=1)

In [ ]:
merged_data['col_6']=merged_data['col_4'].apply(lambda x:x[0])
merged_data['col_7']=merged_data['col_4'].apply(lambda x:x[1])

In [ ]:
merged_data.drop(columns=['col_4','col_5'],inplace=True)

In [ ]:
merged_data.to_excel('final_output.xlsx')

## to get all the link text at once 

with open('links.pickle', 'wb') as handle:
    pickle.dump(all_items, handle, protocol=pickle.HIGHEST_PROTOCOL)

all_items

file = open("links.pickle",'rb')
object_file = pickle.load(file)

driver=webdriver.Chrome('chromedriver.exe')
for item,value in object_file.items():
    for item_child,value_child in value.items():
        try:
            driver.get(value_child)
            time.sleep(5)
            value[item_child]=driver.find_element_by_class_name('main-content').text
        except:
            value[item_child]=''

with open('scapped_data.pickle', 'wb') as handle:
    pickle.dump(object_file, handle, protocol=pickle.HIGHEST_PROTOCOL)

file = open("scapped_data.pickle",'rb')
scarpped_file = pickle.load(file)